<a href="https://colab.research.google.com/github/institutohumai/cursos-python/blob/master/MachineLearning/10_AprendizajeNoSupervisado/ejercicios/ejercicios.ipynb"> <img src='https://colab.research.google.com/assets/colab-badge.svg' /> </a>

## *Instalación/Importación de las librerías*

In [ ]:
# !pip install pyeph

In [ ]:
import pyeph
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from  matplotlib.ticker import FuncFormatter
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import plotly.express as px
import numpy as np


## Obtención y exploración de las bases de datos

In [ ]:
eph = pyeph.get(data="eph", year=2021, period=2, tipo_base='hogar')
canastas = pyeph.get(data="canastas") # canasta basica total y alimentaria
adequi = pyeph.get(data="adulto-equivalente") # adulto equivalente

Imprima las 10 primeras entradas de `eph`, ejecuta el método `.info()` y averigue las columnas disponibles

In [ ]:
# Type your code here:


Recomendamos ampliar la exploración de los datos a lo que considere necesario.

## Reducción a las variables de interés

Vemos columnas de interes en la documentación de la encuentas:

https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_registro_4T2021.pdf

Las columnas de interés son las siguientes:

In [ ]:
columnas = ['REGION', 'MAS_500', 'AGLOMERADO', 'IV1', 'IV2', 'IV3', 'V4',  'IV5', 'IV6', 'IV7',
 'IV8', 'IV9', 'IV10', 'IV11', 'II1', 'II2', 'II3', 'II4_3', 'II7', 'II8', 'II9',
 'ITF', 'IPCF', 'IX_TOT']

Cree una variable llamada `eph_redux` que reduja el dataframe `eph` a las columnas de interés:

In [ ]:
# Type your code here:


## Limpieza de los datos

1. Si durante la exploración fue minucioso habrá notado que la columna `'IPCF'` contiene valores decimales definidos por medio de comas. A fin de mantener una consistencia y facilitar futuras operaciones numéricas, convierta esas comas en puntos decimales.

2. ¿Esta transformación cambia el tipo de datos en dicha columna a `float`? Si no es así, conviértala a `float`.

3. Si exploró la columna `'MAS_500'` y leyó la documentación, sabrá que los únicos valores existentes son:

    &emsp; N ->  Conjunto de aglomerados de menos de 500.000 habitantes

    &emsp; S -> Conjunto de aglomerados de 500.000 y más habitantes

    Reemplace los valores como se indica a continuación para representarlos como booleanos:

    &emsp; N -> 0

    &emsp; S -> 1

4. Chequee por medio del método `.info()` que todas las columnas salvo `'IPCF'` sean de tipo `int64`.

In [ ]:
# Type your code here:


## Gráficas de interés

Graficamos algunas distribuciones del DataFrame `eph_redux` empleando la librería Seaborn:

1. Un histograma que muestra la distribución de valores en la columna `'IPCF'`.

2. Un histograma que muestra la distribución de valores en la columna `'IV1'`.

3. Una visualización de relaciones bivariadas entre las siguientes columnas: `'II8'`, `'II9'`, `'ITF'`, `'IPCF'`, `'IX_TOT'` y `'IV1'` (utilice la función `pairplot`). Coloree los puntos de los gráficos de dispersión según los valores de la columna `'IV1'` (este procedimiento nos permitirá entender cómo la variable 'IV1' se relaciona con las demás).

In [ ]:
# Type your code here:


## Reducción de dimensionalidades

Suponiendo que tenemos conjuntos convexos podemos reducir dimensionalidad con PCA.

Pero utilizaremos otros métodos menos estudiados:

In [ ]:
# !pip install umap-learn

In [ ]:
import umap

El dataset `eph_redux` contiene diversos indicadores socioeconómicos que pueden variar en magnitud, unidad y rango. Para muchos algoritmos de Machine Learning, es crucial que todas las características tengan la misma escala para que el modelo funcione correctamente.

**Objetivo:** Llevar a cabo una normalización standard de las características y luego aplicar la técnica de reducción de dimensionalidad UMAP para obtener proyecciones en 2D y 3D.

1. Instancie un objeto `StandardScaler` y utilice el método `fit_transform` para normalizar todas las columnas del dataset `eph_redux` utilizando Scikit-learn. Debes obtener un nuevo conjunto de datos llamado `eph_redux_scaled`.

    **Reminder:** La normalización standard ajustará cada característica para que tenga una media de 0 y una desviación estándar de 1.

2. Instancie dos objetos de la clase UMAP: uno con 2 componentes llamado `reducer_2d` y otro con 3 componentes llamado `reducer_3d`.

3. Usa el objeto `reducer_2d` para obtener una proyección en 2D de `eph_redux_scaled`. Guárdala en una variable llamada `embedding_2d`.

4. Usa el objeto `reducer_3d` para obtener una proyección en 3D de `eph_redux_scaled`. Guárdala en una variable llamada `embedding_3d`.

5. Verifica las dimensiones de `embedding_2d` y `embedding_3d` para asegurarte de que tienes una representación con las dimensiones correctas.

In [ ]:
# Type your code here:


Si has llamado correctamente a las variables, las siguientes celdas plotean los resultados de la reducción dimensional.

**En 2D:**

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(embedding_2d[:, 0], embedding_2d[:, 1], cmap='Spectral', s=5)
plt.colorbar(boundaries=np.arange(11)-0.5).set_ticks(np.arange(10))
plt.title('UMAP projection of the EPH dataset (2D)', fontsize=18)
plt.show()

**En 3D:**

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(embedding_3d[:, 0], embedding_3d[:, 1], embedding_3d[:, 2], s=5)
plt.title('UMAP projection of the EPH dataset (3D)', fontsize=18)
plt.show()

## Clustering

Ver métodos de sci-kit learn en:

https://scikit-learn.org/stable/modules/clustering.html

In [ ]:
from sklearn.cluster import AffinityPropagation
from sklearn import metrics
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import homogeneity_score

### Clustering con Affinity Propagation en Datos Reducidos

Dado el conjunto de datos embedding_2d, que representa una proyección 2D de un dataset original de alta dimensión, se desea identificar patrones o agrupaciones naturales en los datos.

1. Utilice el método `AffinityPropagation` de Scikit-learn para llevar a cabo una técnica de clustering en `embedding_2d`. Ajuste el parámetro `preference` a -50 y establezca un `random_state` de 0 para garantizar la reproducibilidad.

2. Determine el número estimado de clusters obtenido.

3. Calcule el coeficiente de silueta para evaluar la calidad de los clusters formados. Use la métrica `"sqeuclidean"` para esta tarea.

4. Imprima el número estimado de clusters y el coeficiente de Silhouette.

**Nota:** El coeficiente de Silhouette (silueta) es una métrica que mide qué tan similares son los objetos dentro de su propio cluster en comparación con otros clusters. Los valores más altos indican una mejor definición de clusters.

In [ ]:
# Type your code here:


Se tomó su tiempo, eh... Sabes por qué?

El tamaño de `embedding_2d` es de 15,620 muestras con 2 características cada una.

`AffinityPropagation` tiene una complejidad cuadrática respecto al número de muestras. Esto significa que el tiempo de ejecución aumentará drásticamente con la adición de más muestras. Con 15,620 muestras, el algoritmo tiene que procesar en el orden de 
$15620^2$, o más de 244 millones de pares de muestras.

😌

### Clustering con MiniBatchKMeans en Embeddings 2D

Continuemos con el mismo conjunto de datos reducido a 2 dimensiones, `embedding_2d`. La tarea es evaluar el rendimiento de `MiniBatchKMeans` con diferentes números de clusters para identificar la cantidad óptima.

Pasos a seguir:

1. Defina una lista de posibles cantidades de clusters a considerar: 2, 3, 5, 7, 10 y 13, que serán los hiperparámetros del modelo.

2. Inicialice listas vacías `iner_list` y `homo_list` para almacenar la inercia y la homogeneidad de cada configuración, respectivamente.

3. Inicialice un diccionario llamado `estimators` para guardar los modelos `MiniBatchKMeans` con diferentes números de clusters.

4. Para cada cantidad de clusters en la lista:

    * Cree una instancia de `MiniBatchKMeans` con el número actual de clusters.

    * Ajuste el modelo al conjunto de datos `embedding_2d`.

    * Almacene la inercia (`.inertia_`) del modelo en iner_list.

**Nota:** En la solución proporcionada, solo se ha trabajado con la inercia. Si se quisieras trabajar también con la homogeneidad, debería calcularla y agregarla a homo_list en este paso. Lo dejo como ejercicio.

In [ ]:
# Type your code here:


**Visualización de la Inercia:**

Analice `iner_list` para identificar la cantidad de clusters que minimiza la inercia. Utiliza Plotly para graficar cómo varía la inercia con diferentes números de clusters.

* En el eje X, coloca el rango de los números de clusters (`K`).

* En el eje Y, coloca los valores de inercia de la lista `iner_list`.

In [ ]:
# Type your code here:


En las siguientes celdas, tomemos uno de las entradas de nuestro diccionario `estimators`; `K=5`. Los valores indican a qué cluster pertenece cada punto.

Se compara con la `.shape` de `embedding_2d`.

In [ ]:
estimators[5].labels_.reshape(-1, 1).shape

In [ ]:
embedding_2d.shape

**Preparación de datos para la visualización:**

Crea un DataFrame llamado `data` que contenga las coordenadas de `embedding_2d` y las etiquetas del modelo con 5 clusters. 

Las columnas podrían llamarse `'x'`, `'y'` y `'cluster'` respectivamente.

Imprima las 5 primeras entradas del dataframe.

In [ ]:
# Type your code here:


**Visualización de los Clusters:**

Utilizando Plotly, crea un gráfico de dispersión de los datos en `data`.

Usa las columnas `'x'` e `'y'` para los ejes X e Y respectivamente.

Colorea los puntos según la columna `'cluster'` para distinguir entre los diferentes clusters.

In [ ]:
# Type your code here:
